### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *


C:\Users\abiga\OneDrive\Documents\PythonScripts\llm_eval\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [2]:
models_dict = {
    'claude-2.1':  "ANTHROPIC", 
    'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
num_runs = 2
temperature = .9
num_perturbations= 1
is_file_path = True
llm_evaluation_model = ['gpt-4', "OPENAI"]
instructions = "Please answer thoroughly: "
perturbation_model = ['gpt-3.5-turbo-0301', "OPENAI"]

pipeline = LLMAnalysisPipeline(
    input_data=csv_file_path, 
    models_dict=models_dict, 
    perturbation_model=perturbation_model, 
    llm_evaluation_model=llm_evaluation_model,
    temperature = temperature,
    num_runs= num_runs,
    is_file_path = is_file_path,
    similarity_model_name = similarity_model_name,
    num_perturbations = num_perturbations,
    instructions = instructions

)

# Run the pipeline
df_responses = pipeline.run_pipeline()




2023-12-13 15:46:34,564 - INFO - Reading prompts from CSV file.
2023-12-13 15:46:38,872 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:46:41,545 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:46:46,714 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:46:49,573 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:46:51,953 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:46:56,172 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:47:13,753 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:47:14,668 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-13 15:47:15,579 - INFO -

In [3]:
df_responses

,model,original_prompt,response,temperature,actual_prompt,target_answer,keywords,similarity_scores,keyword_scores,llm_ratings
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondrio...,0.9,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondrio...,"[mitochondrion, mitochondria, atp, adenosine t...",0.871755,0.428571,8
1,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,The basic unit of life is the cell. This means...,0.9,What is the basic unit of life and what does t...,The basic unit of life is the cell. This conce...,"[cell, biology, prokaryotic cells, eukaryotic ...",0.921858,0.466667,8
2,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,Genes are segments of DNA molecules which cont...,0.9,What do genes consist of and how do they work?,Genes are segments of DNA (deoxyribonucleic ac...,"[genes, dna, deoxyribonucleic acid, nucleotide...",0.792699,0.692308,10
3,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,The organelle that contains the cell's genetic...,0.9,Which organelle contains the cell's futurogeni...,The Chronocytum is a unique organelle in cells...,"[chronocytum, futurogenic material, temporal a...",0.460168,0.000000,1
4,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,"As an AI language model, I do not have access ...",0.9,What particle stabilizes the membrane of hover...,"In the world of hover frogs, their unique abil...","[levitonium, hover frogs, membrane stabilizati...",0.588004,0.111111,1
5,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,"As an AI language model, I do not have persona...",0.9,How do astroflora perform photosynthesis in lo...,"Astroflora, a unique class of extraterrestrial...","[astroflora, photosynthesis, low light conditi...",0.800584,0.454545,6


## Responses from LLMs

### With Similarity Scores

### Best and Worst Response from Each Model/Prompt Combo

### With Keyword Matching